## 认识价差以及经典指标的测试（2021.12.19）

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import importlib
import sys
sys.path.append('C:/Users/Administrator/Desktop/Repositories/Low-Frequency-Spread-Estimator')


from SpreadEstimator.SpreadEstimator import SpreadEstimator

In [8]:
import SpreadEstimator
importlib.reload(SpreadEstimator.SpreadEstimator)
from SpreadEstimator.SpreadEstimator import SpreadEstimator

In [2]:
se = SpreadEstimator()

#### 任意绘制一个股票的平均日内价差2019年走势

### 简易测试几个随意构造的价差估计器的截面平均corr

In [3]:
stats, signal = se.test_factor('div{high,low}')

mean corr: 0.1182, positive_corr_ratio: 0.9877, corr_IR: 2.3695
